In [21]:
!nvidia-smi

Wed Jul 26 19:37:53 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A6000    On   | 00000000:4F:00.0 Off |                    0 |
| 30%   28C    P8    22W / 300W |      3MiB / 45631MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA RTX A6000    On   | 00000000:52:00.0 Off |                    0 |
| 30%   

In [22]:
%env CUDA_VISIBLE_DEVICES=2
%env TOKENIZERS_PARALLELISM=false


env: CUDA_VISIBLE_DEVICES=2
env: TOKENIZERS_PARALLELISM=false


In [23]:
from transformers import XGLMTokenizer, XGLMForCausalLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_dataset, concatenate_datasets
import evaluate
import numpy as np
import torch
from torch import tensor 
import os
from transformers import DataCollatorForLanguageModeling
from functools import partial
import json

In [24]:
torch.manual_seed(0)
torch.use_deterministic_algorithms(True)
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"

In [25]:
tgt_lang = "ja"
file_path = "/home/sumire/thesis/LLM_Contextual_Prompt_MT/data/iwslt_hf/"
data_files = { "test": f"{file_path}ted_en-{tgt_lang}"}
dataset = load_dataset("json", data_files=data_files)
dataset

Using custom data configuration default-17fa6efc908acb86
Found cached dataset json (/home/sumire/.cache/huggingface/datasets/json/default-17fa6efc908acb86/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab)


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    test: Dataset({
        features: ['talk_id', 'doc'],
        num_rows: 92
    })
})

In [26]:
dataset["test"][0]["doc"]["ja"][15]

'共和国の指導者は何と呼ぶべきでしょう?'

In [27]:
dataset["test"]["talk_id"][0:1]

[1548]

In [28]:
#K-shot Prompt
prompt_talk_id =  1548
k = 3
kshot = ""

for doc in dataset["test"]:
    if doc["talk_id"] == prompt_talk_id:
        print ("Yes")
        for en_sent, tgt_lang_sent in zip(doc["doc"]["en"][:k], doc["doc"][tgt_lang][:k]):
            print (en_sent)
            shot = f"{en_sent} = {tgt_lang_sent} </s> "
            print (shot)
            kshot += shot
print (kshot)

Yes
the Oxford English Dictionary is "snollygoster."
the Oxford English Dictionary is "snollygoster." = オックスフォード英語辞典で </s> 
Just because it sounds so good.
Just because it sounds so good. = お気入りの単語は “snollygoster” です </s> 
And what snollygoster means is  "a dishonest politician."
And what snollygoster means is  "a dishonest politician." = 響きがいいからです “snollygoster” の意味は </s> 
the Oxford English Dictionary is "snollygoster." = オックスフォード英語辞典で </s> Just because it sounds so good. = お気入りの単語は “snollygoster” です </s> And what snollygoster means is  "a dishonest politician." = 響きがいいからです “snollygoster” の意味は </s> 


In [29]:
#K-shot Prompt BLOOM
prompt_talk_id =  1548
k = 3
kshot_bloom = ""

for doc in dataset["test"]:
    if doc["talk_id"] == prompt_talk_id:
        print ("Yes")
        for en_sent, tgt_lang_sent in zip(doc["doc"]["en"][:k], doc["doc"][tgt_lang][:k]):
            print (en_sent)
            shot = f"Translate  English to Japanese: {en_sent}:"
            print (shot)
            kshot_bloom += shot
print (kshot_bloom)

Yes
the Oxford English Dictionary is "snollygoster."
Translate  English to Japanese: the Oxford English Dictionary is "snollygoster.":
Just because it sounds so good.
Translate  English to Japanese: Just because it sounds so good.:
And what snollygoster means is  "a dishonest politician."
Translate  English to Japanese: And what snollygoster means is  "a dishonest politician.":
Translate  English to Japanese: the Oxford English Dictionary is "snollygoster.":Translate  English to Japanese: Just because it sounds so good.:Translate  English to Japanese: And what snollygoster means is  "a dishonest politician.":


In [30]:
inputs = [kshot + sent + ' = ' for doc in dataset["test"]["doc"] for sent in doc["en"] ][2:3] 
print (inputs)
targets = [sent + ' = ' for doc in dataset["test"]["doc"] for sent in doc[tgt_lang] ][2:3] 
print ()

['the Oxford English Dictionary is "snollygoster." = オックスフォード英語辞典で </s> Just because it sounds so good. = お気入りの単語は “snollygoster” です </s> And what snollygoster means is  "a dishonest politician." = 響きがいいからです “snollygoster” の意味は </s> And what snollygoster means is  "a dishonest politician." = ']



In [31]:
import pandas as pd
df= pd.read_json(f"{file_path}ted_en-{tgt_lang}")
df["doc"]
for doc in df["doc"]:
    for sent in doc["en"]:
        print (sent)

the Oxford English Dictionary is "snollygoster."
Just because it sounds so good.
And what snollygoster means is  "a dishonest politician."
Although there was a 19th-century  newspaper editor who defined it rather better when he said,  "A snollygoster is a fellow who seeks office  regardless of party, platform or principle,  and who, when he wins,  gets there by the sheer force  of monumental talknophical assumnancy."
Now I have no idea what "talknophical" is.
Something to do with words, I assume.
But it's very important that words are at the center of politics,  and all politicians know they have to try and control language.
It wasn't until, for example, 1771 that the British Parliament allowed newspapers to report  the exact words that were said in the debating chamber.
And this was actually all down to the bravery  of a guy with the extraordinary name of Brass Crosby,  who took on Parliament.
And he was thrown into the Tower of London  and imprisoned,  but he was brave enough,  he wa

They're homelands of somebody.
They represent branches of the human imagination  that go back to the dawn of time. And for all of us,  the dreams of these children, like the dreams of our own children,  become part of the naked geography of hope.
So, what we're trying to do at the National Geographic, finally,  is, we believe that politicians will never accomplish anything.
We think that polemics --   --  we think that polemics are not persuasive,  but we think that storytelling can change the world,  and so we are probably the best storytelling institution  in the world. We get 35 million hits on our website every month.
156 nations carry our television channel.
Our magazines are read by millions.
And what we're doing is a series of journeys  to the ethnosphere where we're going to take our audience  to places of such cultural wonder  that they cannot help but come away dazzled  embrace gradually, one by one,  that this world deserves to exist in a diverse way,  that we can find a way

CA: Thank you for scaring the living daylights out of us. Thank you, Ralph.
I have spent the past few years  putting myself into situations  that are usually very difficult  and at the same time somewhat dangerous.
I went to prison --  difficult.
I worked in a coal mine --  dangerous.
I filmed in war zones --  difficult and dangerous.
And I spent 30 days eating nothing but this --  fun in the beginning,  little difficult in the middle, very dangerous in the end.
In fact, most of my career,  I've been immersing myself  into seemingly horrible situations  for the whole goal of trying  to examine societal issues  that hopefully break them down in a way  that make them entertaining and accessible to an audience.
So when I knew I was coming here  to do a TED Talk that was going to look at the world of branding and sponsorship,  I knew I would want to do something a little different.
So as some of you may or may not have heard,  a couple weeks ago, I took out an ad on eBay.
I sent out some F

the gesture is of handing your camera to a total stranger.
No one's ever refused, and luckily no one's ever run off with our camera.
Back then, we had no idea how much this trip would change our lives.
It's really become sacred to us.
This one was taken just weeks after 9/11,  and I found myself trying to explain what had happened that day  in ways a five-year-old could understand.
So these photos are far more than proxies  for a single moment, or even a specific trip.
They're also ways for us to freeze time  for one week in October  and reflect on our times  and how we change from year to year,  and not just physically, but in every way.
Because while we take the same photo,  our perspectives change,  and she reaches new milestones,  and I get to see life through her eyes,  and how she interacts with and sees everything.
This very focused time we get to spend together  is something we cherish and anticipate the entire year.
Recently, on one trip, we were walking,  and she stops dead i

To be part of the constitution, you actually have to pass it twice in the congress.
On February 17th they passed it again  with another vote of 114 to one.
Immediately after that vote,  on February 21st to the 24th,  a delegation of about 30 Hondurans  that are most interested in getting into the city building business.
One is South Korea.
This is a picture of a big, new city center  that's being built in South Korea --  bigger than downtown Boston.
Everything you see there was built in four years,  after they spent four years getting the permits.
The other place that's very interested in city building is Singapore.
They've actually built two cities already in China  and are preparing the third.
So if you think about this practically,  here's where we are.
They've got a site; they're already thinking about this site for the second city.
They're putting in place a legal system  that could allow for managers to come in,  and also an external legal system.
One country has already voluntee

...

That means that ballplayer batted safely, hit safely  three times out of 10 at bats.
That means hit the ball into the outfield,  it dropped, it didn't get caught,  and whoever tried to throw it to first base didn't get there in time  and the runner was safe.
Three times out of 10.
Do you know what they call a 300 hitter  in Major League Baseball?
Good, really good,  maybe an all-star.
Do you know what they call  a 400 baseball hitter?
That's somebody who hit, by the way,  four times safely out of every 10.
Legendary --  as in Ted Williams legendary --  the last Major League Baseball player  to hit over 400 during a regular season.
Now let's take this back into my world of medicine  where I'm a lot more comfortable,  or perhaps a bit less comfortable  after what I'm going to talk to you about.
Suppose you have appendicitis  and you're referred to a surgeon  who's batting 400 on appendectomies.
Somehow this isn't working out, is it?
Now suppose you live  in a certain part of a certain re

But there are over 48 hours of video uploaded to YouTube  every minute.
And of that, only a tiny percentage  ever goes viral and gets tons of views and becomes a cultural moment.
So how does it happen?
Three things:  tastemakers, communities of participation  and unexpectedness.
All right, let's go.
Bear Vasquez: Oh, my God. Oh, my God.
Oh, my God!
Wooo!
Ohhhhh, wowwww!
KA: Last year, Bear Vasquez posted this video  that he had shot outside his home in Yosemite National Park.
In 2010, it was viewed 23 million times.
This is a chart of what it looked like  when it first became popular last summer.
But he didn't actually set out to make a viral video, Bear.
He just wanted to share a rainbow.
Because that's what you do when your name is Yosemite Mountain Bear.
And he had posted lots of nature videos in fact.
And this video had actually been posted  all the way back in January.
So what happened here?
Jimmy Kimmel actually.
Jimmy Kimmel posted this tweet  that would eventually propel the vi

It suggests how government could work better --  not more like a private company,  as many people think it should.
And not even like a tech company,  but more like the Internet itself.
And that means permissionless,  it means open, it means generative.
And that's important.
But what's more important about this app  is that it represents how a new generation  is tackling the problem of government --  not as the problem of an ossified institution,  but as a problem of collective action.
because, it turns out, we're very good at collective action  with digital technology.
Now there's a very large community of people  that are building the tools that we need  to do things together effectively.
It's not just Code for America fellows,  there are hundreds of people all over the country  that are standing and writing civic apps every day in their own communities.
They haven't given up on government.
They are frustrated as hell with it,  but they're not complaining about it,  they're fixing it.

And I founded Maysoon's Kids,  a charity that hopes to give Palestinian refugee children  a sliver of the chance my parents gave me.
But the one moment that stands out the most  was when I got -- before this moment --     But the one moment that stands out the most  was when I got to perform  for the man who floats like a butterfly  and stings like a bee,  has Parkinson's and shakes just like me,  Muhammad Ali.
It was the only time that my father ever saw me perform live,  and I dedicate this talk to his memory.
My name is Maysoon Zayid,  and if I can can, you can can.
This may sound strange,  but I'm a big fan of the concrete block.
The first concrete blocks were manufactured in 1868  with a very simple idea:  modules made of cement of a fixed measurement  that fit together.
Very quickly concrete blocks became  the most-used construction unit in the world.
They enabled us to to build things that were larger than us,  buildings, bridges,  one brick at a time.
Essentially concrete block

Zak Ebrahim is not my real name.
I changed it when my family decided to end our connection with my father  and start a new life.
So why would I out myself  and potentially put myself in danger?
Well, that's simple.
I do it in the hopes that perhaps someone someday  who is compelled to use violence  may hear my story and realize  that there is a better way,  that although I had been subjected  to this violent, intolerant ideology,  that I did not become fanaticized.
Instead, I choose to use my experience  to fight back against terrorism,  against the bigotry.
I do it for the victims of terrorism  and their loved ones,  for the terrible pain and loss that terrorism has forced upon their lives.
For the victims of terrorism, I will speak out  against these senseless acts  and condemn my father's actions.
And with that simple fact, I stand here as proof  that violence isn't inherent in one's religion or race,  and the son does not have to follow  the ways of his father.
I am not my father.


In [ ]:
model_checkpoint = "facebook/xglm-1.7B"
#model_checkpoint = "facebook/xglm-2.9B"
#model_checkpoint = "facebook/xglm-4.5B"
#model_checkpoint = "facebook/xglm-7.5B"
#model_checkpoint = "bigscience/mt0-small"
#tokenizer = XGLMTokenizer.from_pretrained(model_checkpoint,padding_side = 'left', max_new_tokens=250, max_length=250, truncation=True, padding='max_length')
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint,padding_side = 'left', truncation=True, padding='max_length', max_new_tokens=250)
model =  XGLMForCausalLM.from_pretrained(model_checkpoint)
#model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
max_length = 128

def preprocess_function(data): # data should be splitted into train / dev / test internally
    
    inputs = [kshot + sent + ' = ' for doc in data["doc"] for sent in doc["en"]][:50]
    #inputs = ["Translate English to Japanese: " + sent for doc in data["doc"] for sent in doc["en"]][:50]
    print (inputs[0])
    targets = [sent for doc in data["doc"] for sent in doc[tgt_lang] ][:50] 
    print (targets[0])
    model_inputs = tokenizer(
        inputs, text_target=targets, max_length=500, truncation=True, padding='max_length'
    )
    #model_inputs["labels"]
    
    #model_inputs = tokenizer(inputs,  max_length=250, truncation=True, padding='max_length')
    #print ("input_ids", model_inputs["input_ids"])
    #print ("labels", model_inputs["labels"])
    
    return model_inputs


loading file sentencepiece.bpe.model from cache at /home/sumire/.cache/huggingface/hub/models--facebook--xglm-1.7B/snapshots/d23a5e8e2164af31a84a26756b9b17f925143050/sentencepiece.bpe.model
loading file tokenizer.json from cache at /home/sumire/.cache/huggingface/hub/models--facebook--xglm-1.7B/snapshots/d23a5e8e2164af31a84a26756b9b17f925143050/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /home/sumire/.cache/huggingface/hub/models--facebook--xglm-1.7B/snapshots/d23a5e8e2164af31a84a26756b9b17f925143050/special_tokens_map.json
loading file tokenizer_config.json from cache at /home/sumire/.cache/huggingface/hub/models--facebook--xglm-1.7B/snapshots/d23a5e8e2164af31a84a26756b9b17f925143050/tokenizer_config.json
loading configuration file config.json from cache at /home/sumire/.cache/huggingface/hub/models--facebook--xglm-1.7B/snapshots/d23a5e8e2164af31a84a26756b9b17f925143050/config.json
Model config XGLMConfig {
  "act

In [37]:
tokenized_datasets = dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=dataset["test"].column_names,
)
# Labels*  2, 6, 48245, 39, 443, 3048, 45469, 45648, 1129, 1338, 30

  0%|          | 0/1 [00:00<?, ?ba/s]

the Oxford English Dictionary is "snollygoster." = オックスフォード英語辞典で </s> Just because it sounds so good. = お気入りの単語は “snollygoster” です </s> And what snollygoster means is  "a dishonest politician." = 響きがいいからです “snollygoster” の意味は </s> the Oxford English Dictionary is "snollygoster." = 
オックスフォード英語辞典で


In [34]:
tokenizer.batch_decode(tokenized_datasets["test"]["labels"], skip_special_tokens=True)

['オックスフォード英語辞典で',
 'お気入りの単語は “snollygoster” です',
 '響きがいいからです “snollygoster” の意味は',
 '“ずる賢い政治家” です 19世紀の新聞編集者の定義はもっと素敵でした 19世紀の新聞編集者の定義はもっと素敵でした “snollygoster は政党 領 主義に かかわらず公職を目指す人で 大いなる卓越した言力によって当選する人のこと” 大いなる卓越した言力によって当選する人のこと” 大いなる卓越した言力によって当選する人のこと”',
 '“言力”って何のことか分かりませんが',
 '言葉と関係あるのでしょう',
 '大事なのは言葉は政治の中核をなし 政治家は言葉を制御しようとする必要を覚えています',
 '例えば イギリス議会が 議事堂で交わされた実際の文言を報道するのを',
 '許可したのは1771年になってからです これが実現したのは ブラス・クロスビー という変わった名前を持つ 勇気ある国会議員のおかげです',
 'クロスビーは犯罪者としてロンドン塔に収監されますが クロスビーは犯罪者としてロンドン塔に収監されますが 勇気を持ち続け 勇気を持ち続けて闘争し 最終的にロンドン市民の支持を得て勝利しました',
 'それからわずか数年後に “as bold as brass”(堂々とした) という表現が初めて使われたのです “as bold as brass”(堂々とした) という表現が初めて使われたのです',
 '多くの人は真から来ていると思っていますが 実は報道の自由のために',
 '奮闘した人の名前なのです さて言葉と政治がどう結びつくか 本当にお見せするには 独立直後のアメリカ合衆国に さかのぼる必要があります',
 '当時直面した問題は 彼らの指導者ジョージ・ワシントンを 何と呼ぶかでした',
 '称号がなかったのです',
 '共和国の指導者は何と呼ぶべきでしょう?',
 'これが議会で長年に渡り議論されたのです',
 'ありとあらゆる提案が出され議論されました ありとあらゆる提案が出され議論されました',
 '例えば ある人たちが提言したのは ワシントン主席執政官 また別の人は ジョージ・ワシントン殿下 さらに別の人は ワシントン アメリカ合衆国国民自由の擁護者',
 '覚えにくい

In [38]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True)

Using mask_token, but it is not set yet.


ValueError: This tokenizer does not have a mask token which is necessary for masked language modeling. You should pass `mlm=False` to train on causal language modeling instead.

In [ ]:
metric1 = evaluate.load("sacrebleu")
metric2 =  evaluate.load("comet")

In [ ]:
def postprocess_text(preds, labels, input_ids):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    input_ids = [[input_id.strip()] for input_id in input_ids]

    return preds, labels, input_ids


In [ ]:
tokenizer.encode(" = ")
tokenizer.encode("=")
tokenizer.encode("= ")

In [ ]:

output_dir = "./results/playingaround"

def compute_metrics(dataset, output_dir, tgt_lang, tokenizer, eval_preds):
    preds, labels, input_ids = eval_preds
    #labels = tokenized_datasets["test"]["labels"]  # Why We have to define it again ?? 
    print ("preds before split:", tokenizer.batch_decode(preds[:5], skip_special_tokens=True))
    #print ("labels:", tokenizer.batch_decode(labels[:5], skip_special_tokens=True))
    print ()
    print ("input before split:",tokenizer.batch_decode(input_ids[:5], skip_special_tokens=True))
    print ()
    
    sep = tokenizer.sep_token_id
    split_id = tokenizer.encode("=")[-1]
    
    # Preds
    if isinstance(preds, tuple):
        preds = preds
    #preds = [ np.array_split(item, np.where(item == sep)[-1])[-1]  for item in preds ]
    preds = [ np.array_split(item, np.where(item == split_id)[-1])[-1]  for item in preds ]
    del_index= [0, 1] # Delete "= " in the beggining of the prediction 
    preds =[ np.delete(item, del_index) for item in preds ]
    
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    print ("splited preds: ", decoded_preds[:5])
    print ()
    print ("splitted preds ids: ", preds)
    print ()
    
    # Store prediction inference
    with open(output_dir+'/translations.txt','w', encoding='utf8') as wf:
         for translation, ids in zip(decoded_preds, preds):
            wf.write(translation.strip()+'\n')
            wf.write(str(ids)+'\n')
    
    
    # Labels
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    #print ("labels from eval_preds", [tokenizer.decode(i, skip_special_tokens=True) for i in labels])
    print ()
    #labels= [ np.array_split(item, np.where(item == sep)[-1])[-1]  for item in labels ]
    #print ("checking labels_token:")
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    print ("splitted labels from dataset: ", decoded_labels[:5])
    print ()
    
    # Input_ids
    #input_ids = np.where(input_ids != -100, input_ids, tokenizer.pad_token_id)
    input_ids = [ np.array_split(item, np.where(item == sep)[-1])[-1]  for item in input_ids ]
    input_ids = [ np.array_split(item, np.where(item == split_id)[-1])[0]  for item in input_ids ] 
    decoded_input_ids = tokenizer.batch_decode(input_ids, skip_special_tokens=True)
    print ("splited input_ids", decoded_input_ids[:5])
    print ()
    

    decoded_preds, decoded_labels, decoded_input_ids = postprocess_text(decoded_preds, decoded_labels, decoded_input_ids)
    
    # bleu
    if tgt_lang == "ja":
        bleu = metric1.compute(predictions=decoded_preds, references=decoded_labels, tokenize='ja-mecab')
    else: 
        bleu = metric1.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": bleu["score"]}

    # comet
    #print ("decoded_input_ids:",  [item for decoded_input_id in decoded_input_ids for item in decoded_input_id][:5], "\ndecoded_preds", decoded_preds[:5], "\ndecoded_labels", [item for decoded_label in decoded_labels for item in decoded_label][:5])
    
    comet = metric2.compute(predictions=decoded_preds, references=[item for decoded_label in decoded_labels for item in decoded_label], sources = [item for decoded_input_id in decoded_input_ids for item in decoded_input_id])
    print ("comet")
    print ("decoded_preds", decoded_preds)
    print()
    print ("references", [item for decoded_label in decoded_labels for item in decoded_label])
    print()
    print ("source",  [item for decoded_input_id in decoded_input_ids for item in decoded_input_id])
    result["comet"] =  np.mean(comet["scores"])
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    print(result)

    # Store the score
    with open(output_dir+'/test_score.txt','w', encoding='utf8') as wf:
        for key, value in result.items():
            wf.write(f"{key}: {value}\n") #ensure_ascii=False

    return result


In [ ]:

training_args = Seq2SeqTrainingArguments(
    output_dir="./results/xglm",
    evaluation_strategy="steps",
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_eval_batch_size=4, 
    do_predict=True,
    do_eval = True,
    predict_with_generate=True,
    include_inputs_for_metrics=True,
    warmup_steps = 500,
    #metric_for_best_model = "comet"
    #load_best_model_at_end = True,
    greater_is_better = True,
    #save_strategy = "steps",
    eval_delay = 0.0,
    eval_accumulation_steps = 20
    
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    #train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    compute_metrics=partial(compute_metrics, dataset, output_dir, tgt_lang, tokenizer),
    data_collator=data_collator,
    
)

trainer.evaluate()


Findings 
- To translate, specific prompt template is needed: "{Source sentence} = {Target sentence}", otherwise performance  
- zero shot is not working
- From one shot it shows some correct translation (but with only a token)
- We can split the prompt and prediction, also input, by "="

Question 
- Input length of input_ids is 500, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
- Padding and Truncation?
- Labels are not handed over to compute_metrics function, inside the eval_preds (At the moment I directly manually put the reference from dataset)
- Why the model predits only one token ? (But the token is good one)